In [51]:
pip install --ignore-installed -r requirements.txt

  Using cached langchain-0.1.19-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached chromadb-0.5.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.30-cp312-cp312-macosx_10_9_x86_64.whl.metadata (9.6 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_10_9_x86_64.whl.metadata (7.5 kB)
  Using cached dataclasses_json-0.6.5-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached langsmith-0.1.56-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_10_9_x86_64.whl.metadata (61 kB)
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.3.0-py3-none-an

# Load project files into documents

In [129]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import DirectoryLoader

project_dir = "/Users/htrinter/workspace/Open-Multiple-URLs/Open-Multiple-URLs/src/"
file_extensions = [".vue", ".ts"]

docs = []
for extension in file_extensions:
    loader = DirectoryLoader(project_dir, glob=f"**/*{extension}", loader_cls=TextLoader, recursive=True)
    docs.extend(loader.load())

print(f"loaded {len(docs)} docs")

loaded 15 docs


# Split documents into chunks

In [130]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

#all_splits = [text_splitter.split_documents(text) for text in text_data]
#all_splits_flat = [chunk for sublist in all_splits for chunk in sublist]

# Generate embeddings from chunks

In [132]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

embeddings = OllamaEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Ask a question about your code

In [133]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA


ollama = Ollama()
qachain = RetrievalQA.from_chain_type(ollama, retriever=vectorstore.as_retriever())

# What does the flag isStoredValuesLoaded do?
q = "I want to implement an option to open links in a new window instead of a new tab. what files would i have to modify to implement this?"
qachain.invoke({ "query" : q })

{'query': 'I want to implement an option to open links in a new window instead of a new tab. what files would i have to modify to implement this?',
 'result': 'To implement an option to open links in a new window instead of a new tab, you will need to modify the following files:\n\n1. `main.js`: In this file, you will need to add a new option to the `urls` array that specifies whether the link should be opened in a new tab or window. You can do this by adding a new property to the `urls` object, such as `windowOpen: false`, and then modifying the `handleLinkClick` function to check this property before opening the link in a new tab.\n2. `link-component.vue`: In this file, you will need to modify the `abbr` component to add a new attribute that specifies whether the link should be opened in a new tab or window. You can do this by adding an `openInNewWindow` attribute to the `abbr` component, and then modifying the template to check this attribute before opening the link in a new tab.\n3